# **PROJEKT-DETEKCIJA LAŽNIH VIJESTI**

### UVOD
U ovom projektu pokušavat ćemo detektirati lažne vijesti(binarna klasifikacija) pri čemu će vrijediti:

*   0-istinita vijest
*   1-lažna vijest

Ideja je bila isprobavavati različite modele(drugačiji hiperparametri i parametri,skriveni slojevi,optimizatori,funkcije gubitka...).Budući da se radi u tekstu,početna ideja bila je usporediti unaprijedne i rekurentne neuronske mreže.Naknadno se pokazalo da su konvolucijske neuronske mreže jako pogodne za ovaj problem(1D konvolucija) dok su rekurentne neuronske mreže bile jako zahtjevne za treniranje s mojim resursima.

Dataset koji ćemo koristiti preuzet je s Kagglea i zove se WELFake_Dataset koji se sastoji od 35028 točnih i 37106 lažnih vijesti.Možemo primjetiti da je otprilike jednako vijesti koje su lažne i istinite.






###POTREBNE BIBLIOTEKE

In [1]:
import zipfile
import pandas as pd
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


###UČITAVANJE I PRIPREMA PODATAKA

Prvo nam je cilj učitati podatke i vidjeti kako oni izgledaju.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/PROJEKT-ANM

/content/drive/MyDrive/PROJEKT-ANM


In [4]:
zip_file_path = 'WELFake_Dataset.csv.zip'

# Putanja do direktorija u koji želite ekstraktirati sadržaj zip datoteke
output_directory = '/content/Drive/MyDrive/PROJEKT-ANM'

# Provjerite je li datoteka prisutna
if os.path.exists(zip_file_path):
    # Stvorite odredišnu mapu ako ne postoji
    os.makedirs(output_directory, exist_ok=True)

    # Ekstraktirajte sadržaj zip datoteke
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_directory)

    # Prikazujete sadržaj odredišne mape
    extracted_files = os.listdir(output_directory)
    print(extracted_files)
else:
    print(f"Datoteka {zip_file_path} nije pronađena.")

['WELFake_Dataset.csv']


In [5]:
csv_file_path = os.path.join(output_directory, 'WELFake_Dataset.csv')

# Učitajte CSV datoteku u Pandas DataFrame
podaci = pd.read_csv(csv_file_path)

In [6]:
podaci

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


In [7]:
podaci.shape

(72134, 4)

In [8]:
podaci.iloc[0] #ispis prvog retka

Unnamed: 0                                                    0
title         LAW ENFORCEMENT ON HIGH ALERT Following Threat...
text          No comment is expected from Barack Obama Membe...
label                                                         1
Name: 0, dtype: object

In [9]:
podaci[podaci['label']==0]

,Unnamed: 0,title,text,label
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
11,11,"May Brexit offer would hurt, cost EU citizens ...",BRUSSELS (Reuters) - British Prime Minister Th...,0
12,12,Schumer calls on Trump to appoint official to ...,"WASHINGTON (Reuters) - Charles Schumer, the to...",0
14,14,No Change Expected for ESPN Political Agenda D...,As more and more sports fans turn off ESPN to ...,0
15,15,Billionaire Odebrecht in Brazil scandal releas...,RIO DE JANEIRO/SAO PAULO (Reuters) - Billionai...,0
...,...,...,...,...
72124,72124,An Unlikely Contender Rises in France as the A...,"PARIS — In the age of Donald J. Trump, “Bre...",0
72126,72126,Determined to kill: Can tough gun laws end mas...,The flag at Desert Hot Springs' Condor Gun Sho...,0
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0


Imamo 35208 istinitih vijesti.

In [10]:
podaci[podaci['label']==1]

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1
...,...,...,...,...
72125,72125,WOW! JILL STEIN’S ‘FIRESIDE CHAT’ Exposes Her ...,,1
72127,72127,WIKILEAKS EMAIL SHOWS CLINTON FOUNDATION FUNDS...,An email released by WikiLeaks on Sunday appea...,1
72128,72128,JUDGE JEANINE SOUNDS FREE SPEECH ALARM: “They ...,Judge Jeanine lets it rip! She s concerned wit...,1
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1


Imamo 37106 lažnih vijesti.

###PROVJERA IMA LI NULL VRIJEDNOSTI

---



In [11]:
null_count = podaci.isnull().sum()


In [12]:
null_count

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

Imamo jako malo nul vrijednosti što je zanemarivo s obzirom na ukupan broj podataka pa ćemo izbaciti sve retke s null vrijednostima.

In [13]:
podaci=podaci.dropna()

In [14]:
podaci

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1
...,...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


###IZBACIVANJE PRVOG STUPCA

In [15]:
podaci = podaci.drop(podaci.columns[:1], axis=1)


In [16]:
podaci

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1
...,...,...,...
72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0


In [17]:
podaci['title']

0        LAW ENFORCEMENT ON HIGH ALERT Following Threat...
2        UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3        Bobby Jindal, raised Hindu, uses story of Chri...
4        SATAN 2: Russia unvelis an image of its terrif...
5        About Time! Christian Group Sues Amazon and SP...
                               ...                        
72129    Russians steal research on Trump in hack of U....
72130     WATCH: Giuliani Demands That Democrats Apolog...
72131    Migrants Refuse To Leave Train At Refugee Camp...
72132    Trump tussle gives unpopular Mexican leader mu...
72133    Goldman Sachs Endorses Hillary Clinton For Pre...
Name: title, Length: 71537, dtype: object

Smanjujemo broj podataka jer je postupak tokenizacije jako memorijski zahtjevan.

In [18]:
podaci_labela_0 = podaci[podaci['label'] == 0]
podaci_labela_1 = podaci[podaci['label'] == 1]

# Uzimamo polovicu podataka iz svake klase
uzorak_labela_0 = podaci_labela_0.sample(n=17500, random_state=42)
uzorak_labela_1 = podaci_labela_1.sample(n=17500, random_state=42)

# Kombinirajte uzorke u novi skup podataka
podaci = pd.concat([uzorak_labela_0, uzorak_labela_1])

###TOKENIZACIJA,UKLANJANJE STOP RIJEČI TE NEPOTREBNIH ZNAKOVA

In [19]:
import nltk

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Preuzimanje stop-riječi iz NLTK i scikit-learn
nltk.download('stopwords')
stop_words_nltk = set(stopwords.words('english'))
stop_words_sklearn = set(ENGLISH_STOP_WORDS)

def preprocess_text(text):
    # Tokenizacija
    tokens = nltk.word_tokenize(text)

    # Uklanjanje nepotrebnih znakova i normalizacija teksta
    tokens = [re.sub(r'[^a-zA-Z0-9]', '', token.lower()) for token in tokens]#uklanja znakove osim slova i brojeva

    # Uklanjanje stop-riječi
    tokens = [token for token in tokens if token not in stop_words_nltk and token not in stop_words_sklearn]

    # Ponovno sastavljanje teksta od tokena
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Primjer primjene funkcije na stupce "text" i "title" u DataFrameu df

podaci['text'] = podaci['text'].apply(preprocess_text)
podaci['title'] = podaci['title'].apply(preprocess_text)
print(podaci)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                                   title  \
42025  people die eating rawmilk cheese new york stat...   
56304  dup blames sinn fein northern ireland talks co...   
23174  exclusive  brazil prosecutor says successor  p...   
49262  intelligence chief says russia involvement 201...   
19935  macri coalition poised win key argentina midte...   
...                                                  ...   
71185  intelligence says details russian dossier expo...   
16072  justice department  firing workers expired pap...   
7810   clinton loses heckler fingerpointing rant   de...   
42619  trump lied  spontaneous  taiwan  planning talk...   
63277  sarah palin loses sh  rage whines hillary char...   

                                                    text  label  
42025  people died following outbreak listeria linked...      0  
56304  belfast  reuters   senior member northern irel...      0  
23174  brasilia  reuters   senior brazilian law enfor...      0  
49262  washingt

In [21]:
podaci

,title,text,label
42025,people die eating rawmilk cheese new york stat...,people died following outbreak listeria linked...,0
56304,dup blames sinn fein northern ireland talks co...,belfast reuters senior member northern irel...,0
23174,exclusive brazil prosecutor says successor p...,brasilia reuters senior brazilian law enfor...,0
49262,intelligence chief says russia involvement 201...,washington reuters director national intell...,0
19935,macri coalition poised win key argentina midte...,buenos aires reuters argentine president ma...,0
...,...,...,...
71185,intelligence says details russian dossier expo...,bombshell dossier exposing donald trump shady ...,1
16072,justice department firing workers expired pap...,justice department control 6 months extra let ...,1
7810,clinton loses heckler fingerpointing rant de...,,1
42619,trump lied spontaneous taiwan planning talk...,according washington post sources close taiwa...,1


###VEKTORIZACIJA PODATAKA

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse


podaci['combined_text'] = podaci['text'] + ' ' + podaci['title']

# Preprocesiranje i vektorizacija
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), min_df=5, max_df=0.9, sublinear_tf=True)

X_tfidf = tfidf_vectorizer.fit_transform(podaci['combined_text'])

y = podaci['label']
X_tfidf_dense = X_tfidf.toarray() if scipy.sparse.issparse(X_tfidf) else X_tfidf


In [23]:
X_tfidf

<35000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 5198472 stored elements in Compressed Sparse Row format>

In [24]:
X_tfidf[0]

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 204 stored elements in Compressed Sparse Row format>

In [ ]:
del X_tfidf
del podaci

###PODJELA NA SKUP ZA TRENIRANJE I TESTIRANJE

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf_dense, y, test_size=0.2, random_state=42)


In [ ]:
X_train.shape

(28000, 5000)

###**UNAPRIJEDNE NEURONSKE MREŽE**

Krećemo s izradom modela unaprijednih neuronskih mreža.Isprobavat ćemo razne slučajeve:različite brojeve neurona,skrivenih slojeva,epoha,batch_size,drugačije optimizatore i funkcije gubitka te vidjeti koji nam je najbolji krajnji model.


###**1.POKUŠAJ**:2 skrivena sloja,10 epoha,batch_size=32,bez regularizacije

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128,input_shape=(X_train.shape[1],),activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=10,batch_size=32)


Epoch 1/10
875/875 [==============================] - 9s 9ms/step - loss: 0.1894 - accuracy: 0.9271
Epoch 2/10
875/875 [==============================] - 7s 8ms/step - loss: 0.0911 - accuracy: 0.9682
Epoch 3/10
875/875 [==============================] - 8s 9ms/step - loss: 0.0573 - accuracy: 0.9804
Epoch 4/10
875/875 [==============================] - 6s 7ms/step - loss: 0.0261 - accuracy: 0.9904
Epoch 5/10
875/875 [==============================] - 7s 9ms/step - loss: 0.0139 - accuracy: 0.9954
Epoch 6/10
875/875 [==============================] - 12s 14ms/step - loss: 0.0060 - accuracy: 0.9987
Epoch 7/10
875/875 [==============================] - 6s 7ms/step - loss: 0.0048 - accuracy: 0.9989
Epoch 8/10
875/875 [==============================] - 8s 9ms/step - loss: 0.0040 - accuracy: 0.9989
Epoch 9/10
875/875 [==============================] - 6s 7ms/step - loss: 0.0036 - accuracy: 0.9987
Epoch 10/10
875/875 [==============================] - 7s 8ms/step - loss: 0.0024 - accuracy: 0.99

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 1s 2ms/step - loss: 0.2641 - accuracy: 0.9547


[0.264142781496048, 0.954714298248291]

Vidimo da smo dobili jako veliku točnost.

###**2**.**POKUŠAJ**:2 skrivena sloja,10 epoha,batch_size=32,regularizacija

In [ ]:
from tensorflow.keras.layers import Dropout

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=10,batch_size=32)


Epoch 1/10
875/875 [==============================] - 7s 7ms/step - loss: 0.2196 - accuracy: 0.9142
Epoch 2/10
875/875 [==============================] - 7s 9ms/step - loss: 0.0957 - accuracy: 0.9674
Epoch 3/10
875/875 [==============================] - 6s 7ms/step - loss: 0.0608 - accuracy: 0.9799
Epoch 4/10
875/875 [==============================] - 7s 8ms/step - loss: 0.0395 - accuracy: 0.9877
Epoch 5/10
875/875 [==============================] - 7s 8ms/step - loss: 0.0269 - accuracy: 0.9916
Epoch 6/10
875/875 [==============================] - 7s 8ms/step - loss: 0.0192 - accuracy: 0.9941
Epoch 7/10
875/875 [==============================] - 6s 7ms/step - loss: 0.0137 - accuracy: 0.9960
Epoch 8/10
875/875 [==============================] - 7s 8ms/step - loss: 0.0121 - accuracy: 0.9964
Epoch 9/10
875/875 [==============================] - 6s 7ms/step - loss: 0.0098 - accuracy: 0.9975
Epoch 10/10
875/875 [==============================] - 7s 8ms/step - loss: 0.0067 - accuracy: 0.9979

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 1s 2ms/step - loss: 0.2248 - accuracy: 0.9563


[0.22476930916309357, 0.9562857151031494]

Vidimo da je točnost nešto bolja,a gubitak manji.Ubuduće ćemo koristiti regularizaciju.
###**3.POKUŠAJ**:2 skrivena sloja,10 epoha,batch_size=32,optimizator sgd


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=10,batch_size=32)


Epoch 1/10
875/875 [==============================] - 4s 4ms/step - loss: 0.6913 - accuracy: 0.5408
Epoch 2/10
875/875 [==============================] - 5s 6ms/step - loss: 0.6828 - accuracy: 0.6144
Epoch 3/10
875/875 [==============================] - 4s 4ms/step - loss: 0.6639 - accuracy: 0.6813
Epoch 4/10
875/875 [==============================] - 3s 4ms/step - loss: 0.6227 - accuracy: 0.7458
Epoch 5/10
875/875 [==============================] - 6s 7ms/step - loss: 0.5423 - accuracy: 0.7983
Epoch 6/10
875/875 [==============================] - 3s 4ms/step - loss: 0.4433 - accuracy: 0.8397
Epoch 7/10
875/875 [==============================] - 4s 4ms/step - loss: 0.3626 - accuracy: 0.8651
Epoch 8/10
875/875 [==============================] - 4s 5ms/step - loss: 0.3064 - accuracy: 0.8898
Epoch 9/10
875/875 [==============================] - 4s 5ms/step - loss: 0.2687 - accuracy: 0.9011
Epoch 10/10
875/875 [==============================] - 4s 5ms/step - loss: 0.2354 - accuracy: 0.9154

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 1s 2ms/step - loss: 0.1894 - accuracy: 0.9289


[0.1893695890903473, 0.928857147693634]

Točnost je i dalje visoka,ali ipak nešto niža nego s optimizatorom adam.S druge strane,gubitak je manji.

###**4.POKUŠAJ**:2 skrivena sloja,batch_size=32,broj epoha 20,optimizator adam

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=32)


Epoch 1/20
875/875 [==============================] - 8s 8ms/step - loss: 0.2226 - accuracy: 0.9153
Epoch 2/20
875/875 [==============================] - 6s 7ms/step - loss: 0.0987 - accuracy: 0.9645
Epoch 3/20
875/875 [==============================] - 8s 9ms/step - loss: 0.0657 - accuracy: 0.9782
Epoch 4/20
875/875 [==============================] - 6s 7ms/step - loss: 0.0405 - accuracy: 0.9871
Epoch 5/20
875/875 [==============================] - 7s 8ms/step - loss: 0.0247 - accuracy: 0.9921
Epoch 6/20
875/875 [==============================] - 6s 7ms/step - loss: 0.0174 - accuracy: 0.9948
Epoch 7/20
875/875 [==============================] - 7s 8ms/step - loss: 0.0151 - accuracy: 0.9955
Epoch 8/20
875/875 [==============================] - 6s 7ms/step - loss: 0.0148 - accuracy: 0.9955
Epoch 9/20
875/875 [==============================] - 7s 8ms/step - loss: 0.0104 - accuracy: 0.9970
Epoch 10/20
875/875 [==============================] - 6s 7ms/step - loss: 0.0113 - accuracy: 0.9967

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 1s 2ms/step - loss: 0.2453 - accuracy: 0.9591


[0.24530182778835297, 0.9591428637504578]

Točnost nešto veća nego kad imamo 10 epoha i optimizator adam,ali gubitak je nešto veći pa ćemo dalje raditi s 10 epoha zbog uštede vremena i neznatne razlike u točnosti(nakon što provjerimo za sgd slučaj).
###**5.POKUŠAJ**:2 skrivena sloja,batch_size=32,broj epoha 20,optimizator sgd

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=32)


Epoch 1/20
875/875 [==============================] - 6s 6ms/step - loss: 0.6906 - accuracy: 0.5474
Epoch 2/20
875/875 [==============================] - 4s 4ms/step - loss: 0.6808 - accuracy: 0.6295
Epoch 3/20
875/875 [==============================] - 4s 4ms/step - loss: 0.6592 - accuracy: 0.6934
Epoch 4/20
875/875 [==============================] - 5s 6ms/step - loss: 0.6117 - accuracy: 0.7570
Epoch 5/20
875/875 [==============================] - 4s 4ms/step - loss: 0.5246 - accuracy: 0.8085
Epoch 6/20
875/875 [==============================] - 4s 4ms/step - loss: 0.4270 - accuracy: 0.8418
Epoch 7/20
875/875 [==============================] - 4s 5ms/step - loss: 0.3518 - accuracy: 0.8715
Epoch 8/20
875/875 [==============================] - 4s 5ms/step - loss: 0.2969 - accuracy: 0.8935
Epoch 9/20
875/875 [==============================] - 4s 4ms/step - loss: 0.2589 - accuracy: 0.9083
Epoch 10/20
875/875 [==============================] - 4s 4ms/step - loss: 0.2286 - accuracy: 0.9179

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 1s 2ms/step - loss: 0.1203 - accuracy: 0.9544


[0.12028557807207108, 0.9544285535812378]

Točnost neznatno manja,ali je gubitak jako mali.

###**6.POKUŠAJ**:2 skrivena sloja,batch_size=64,broj epoha 20,optimizator sgd

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=64)


Epoch 1/20
438/438 [==============================] - 3s 6ms/step - loss: 0.6918 - accuracy: 0.5336
Epoch 2/20
438/438 [==============================] - 4s 8ms/step - loss: 0.6882 - accuracy: 0.5839
Epoch 3/20
438/438 [==============================] - 3s 6ms/step - loss: 0.6823 - accuracy: 0.6213
Epoch 4/20
438/438 [==============================] - 4s 9ms/step - loss: 0.6742 - accuracy: 0.6553
Epoch 5/20
438/438 [==============================] - 3s 7ms/step - loss: 0.6620 - accuracy: 0.6896
Epoch 6/20
438/438 [==============================] - 3s 8ms/step - loss: 0.6433 - accuracy: 0.7245
Epoch 7/20
438/438 [==============================] - 3s 7ms/step - loss: 0.6150 - accuracy: 0.7570
Epoch 8/20
438/438 [==============================] - 3s 6ms/step - loss: 0.5764 - accuracy: 0.7837
Epoch 9/20
438/438 [==============================] - 3s 6ms/step - loss: 0.5290 - accuracy: 0.8056
Epoch 10/20
438/438 [==============================] - 3s 6ms/step - loss: 0.4778 - accuracy: 0.8270

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 1s 2ms/step - loss: 0.1790 - accuracy: 0.9330


[0.17902934551239014, 0.9330000281333923]

Točnost nam se nešto smanjila,bolje je ići s batch_size=32

###**7.POKUŠAJ**:3 skrivena sloja,batch_size=32,broj epoha 20,optimizator sgd



In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=32)


Epoch 1/20
875/875 [==============================] - 8s 8ms/step - loss: 0.6925 - accuracy: 0.5174
Epoch 2/20
875/875 [==============================] - 7s 9ms/step - loss: 0.6891 - accuracy: 0.5583
Epoch 3/20
875/875 [==============================] - 6s 7ms/step - loss: 0.6799 - accuracy: 0.6084
Epoch 4/20
875/875 [==============================] - 7s 9ms/step - loss: 0.6518 - accuracy: 0.6787
Epoch 5/20
875/875 [==============================] - 6s 7ms/step - loss: 0.5650 - accuracy: 0.7664
Epoch 6/20
875/875 [==============================] - 7s 8ms/step - loss: 0.4200 - accuracy: 0.8410
Epoch 7/20
875/875 [==============================] - 7s 7ms/step - loss: 0.3171 - accuracy: 0.8809
Epoch 8/20
875/875 [==============================] - 7s 8ms/step - loss: 0.2575 - accuracy: 0.9082
Epoch 9/20
875/875 [==============================] - 7s 8ms/step - loss: 0.2214 - accuracy: 0.9225
Epoch 10/20
875/875 [==============================] - 7s 9ms/step - loss: 0.1911 - accuracy: 0.9334

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 1s 4ms/step - loss: 0.1138 - accuracy: 0.9596


[0.11384072154760361, 0.9595714211463928]

Vidimo da je jedan skriveni sloj više popravio točnost modela(ako ga usporedimo s modelima u kojima smo samo promijenili taj parametar).Gubitak se još dodatno smanjio.
###**8.POKUŠAJ**:3 skrivena sloja,batch_size=32,broj epoha 20,optimizator adam


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=32)


Epoch 1/20
875/875 [==============================] - 14s 15ms/step - loss: 0.2175 - accuracy: 0.9114
Epoch 2/20
875/875 [==============================] - 13s 15ms/step - loss: 0.0974 - accuracy: 0.9653
Epoch 3/20
875/875 [==============================] - 14s 15ms/step - loss: 0.0563 - accuracy: 0.9816
Epoch 4/20
875/875 [==============================] - 13s 15ms/step - loss: 0.0321 - accuracy: 0.9894
Epoch 5/20
875/875 [==============================] - 13s 15ms/step - loss: 0.0205 - accuracy: 0.9937
Epoch 6/20
875/875 [==============================] - 13s 15ms/step - loss: 0.0165 - accuracy: 0.9948
Epoch 7/20
875/875 [==============================] - 13s 15ms/step - loss: 0.0140 - accuracy: 0.9959
Epoch 8/20
875/875 [==============================] - 13s 15ms/step - loss: 0.0115 - accuracy: 0.9966
Epoch 9/20
875/875 [==============================] - 14s 15ms/step - loss: 0.0117 - accuracy: 0.9964
Epoch 10/20
875/875 [==============================] - 13s 15ms/step - loss: 0.008

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 1s 3ms/step - loss: 0.2731 - accuracy: 0.9616


[0.273084431886673, 0.961571455001831]

Ovaj model daje najveću točnost dosad,ali je gubitak veći.
###**9.POKUŠAJ**:3 skrivena sloja,batch_size=32,broj epoha 20,optimizator sgd,aktivacija softmax,


In [ ]:
from tensorflow.keras.layers import Dropout


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='softmax'))

In [ ]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=32)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(32, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


875/875 [==============================] - 10s 11ms/step - loss: nan - accuracy: 0.4988
Epoch 2/20
875/875 [==============================] - 8s 9ms/step - loss: nan - accuracy: 0.5003
Epoch 3/20
875/875 [==============================] - 9s 11ms/step - loss: nan - accuracy: 0.5003
Epoch 4/20
875/875 [==============================] - 9s 11ms/step - loss: nan - accuracy: 0.5003
Epoch 5/20
875/875 [==============================] - 8s 9ms/step - loss: nan - accuracy: 0.5003
Epoch 6/20
875/875 [==============================] - 10s 11ms/step - loss: nan - accuracy: 0.5003
Epoch 7/20
875/875 [==============================] - 9s 10ms/step - loss: nan - accuracy: 0.5003
Epoch 8/20
875/875 [==============================] - 9s 10ms/step - loss: nan - accuracy: 0.5003
Epoch 9/20
875/875 [==============================] - 10s 11ms/step - loss: nan - accuracy: 0.5003
Epoch 10/20
875/875 [==============================] - 10s 11ms/step - loss: nan - accuracy: 0.5003
Epoch 11/20
875/875 [=======

In [ ]:
model.evaluate(X_test, y_test)


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


219/219 [==============================] - 2s 8ms/step - loss: nan - accuracy: 0.4987


[nan, 0.49871429800987244]

Jako loša točnost u ovom slučaju.

###**10.POKUŠAJ**:3 skrivena sloja,batch_size=32,broj epoha 20,optimizator adam,aktivacija softmax,gubitak kategorička krosentropija-probati

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=32)


Epoch 1/20
875/875 [==============================] - 22s 25ms/step - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 2/20
875/875 [==============================] - 18s 20ms/step - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 3/20
875/875 [==============================] - 18s 20ms/step - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 4/20
875/875 [==============================] - 17s 19ms/step - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 5/20
875/875 [==============================] - 17s 19ms/step - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 6/20
875/875 [==============================] - 18s 20ms/step - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 7/20
875/875 [==============================] - 18s 20ms/step - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 8/20
875/875 [==============================] - 21s 24ms/step - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 9/20
875/875 [==============================] - 19s 21ms/step - loss: 0.0000e+00 - accuracy: 0.4997
Epoch 10/20
875/875 [=========================

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 2s 8ms/step - loss: 0.0000e+00 - accuracy: 0.5013


[0.0, 0.50128573179245]

Vidimo da kategorička krosentropija nije dobra ideja u ovom primjeru,već binarna krosentropija i sigmoid.
###**11.POKUŠAJ**:3 skrivena sloja,batch_size=32,broj epoha 20,optimizator sgd,aktivacija sigmoid i tanh u skrivenim slojevima

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256,input_shape=(X_train.shape[1],),activation='tanh'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64,activation='tanh'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=32)


Epoch 1/20
875/875 [==============================] - 10s 10ms/step - loss: 0.6771 - accuracy: 0.6047
Epoch 2/20
875/875 [==============================] - 10s 11ms/step - loss: 0.5802 - accuracy: 0.7933
Epoch 3/20
875/875 [==============================] - 9s 11ms/step - loss: 0.3599 - accuracy: 0.8745
Epoch 4/20
875/875 [==============================] - 9s 10ms/step - loss: 0.2395 - accuracy: 0.9124
Epoch 5/20
875/875 [==============================] - 10s 11ms/step - loss: 0.1894 - accuracy: 0.9301
Epoch 6/20
875/875 [==============================] - 9s 11ms/step - loss: 0.1603 - accuracy: 0.9421
Epoch 7/20
875/875 [==============================] - 11s 12ms/step - loss: 0.1427 - accuracy: 0.9498
Epoch 8/20
875/875 [==============================] - 10s 11ms/step - loss: 0.1294 - accuracy: 0.9538
Epoch 9/20
875/875 [==============================] - 9s 10ms/step - loss: 0.1205 - accuracy: 0.9580
Epoch 10/20
875/875 [==============================] - 9s 10ms/step - loss: 0.1109 - a

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 3s 11ms/step - loss: 0.1516 - accuracy: 0.9463


[0.1516234278678894, 0.9462857246398926]

Visoka točnost,nizak gubitak,ali ipak niži nego s funkcijom relu.
###**12.POKUŠAJ**:3 skrivena sloja,batch_size=32,broj epoha 20,optimizator adam,aktivacija sigmoid i tanh u skrivenim slojevima

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256,input_shape=(X_train.shape[1],),activation='tanh'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64,activation='tanh'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=32)


Epoch 1/20
875/875 [==============================] - 18s 19ms/step - loss: 0.1948 - accuracy: 0.9248
Epoch 2/20
875/875 [==============================] - 16s 19ms/step - loss: 0.1074 - accuracy: 0.9627
Epoch 3/20
875/875 [==============================] - 17s 20ms/step - loss: 0.0877 - accuracy: 0.9687
Epoch 4/20
875/875 [==============================] - 17s 20ms/step - loss: 0.0761 - accuracy: 0.9731
Epoch 5/20
875/875 [==============================] - 17s 19ms/step - loss: 0.0641 - accuracy: 0.9782
Epoch 6/20
875/875 [==============================] - 16s 19ms/step - loss: 0.0611 - accuracy: 0.9786
Epoch 7/20
875/875 [==============================] - 17s 19ms/step - loss: 0.0531 - accuracy: 0.9822
Epoch 8/20
875/875 [==============================] - 17s 19ms/step - loss: 0.0481 - accuracy: 0.9845
Epoch 9/20
875/875 [==============================] - 19s 22ms/step - loss: 0.0457 - accuracy: 0.9851
Epoch 10/20
875/875 [==============================] - 16s 19ms/step - loss: 0.041

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 3s 10ms/step - loss: 0.2892 - accuracy: 0.9430


[0.2892393171787262, 0.9430000185966492]

Slično kao u prethodnom primjeru,samo dosta veći gubitak.

###**13.POKUŠAJ**:4 skrivena sloja,batch_size=32,broj epoha 20,optimizator sgd,aktivacija sigmoid i relu u skrivenim slojevima

In [ ]:
from tensorflow.keras.layers import Dropout


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=32)


Epoch 1/20
875/875 [==============================] - 10s 10ms/step - loss: 0.6925 - accuracy: 0.5183
Epoch 2/20
875/875 [==============================] - 9s 10ms/step - loss: 0.6905 - accuracy: 0.5361
Epoch 3/20
875/875 [==============================] - 9s 10ms/step - loss: 0.6842 - accuracy: 0.5798
Epoch 4/20
875/875 [==============================] - 9s 11ms/step - loss: 0.6557 - accuracy: 0.6606
Epoch 5/20
875/875 [==============================] - 12s 13ms/step - loss: 0.5300 - accuracy: 0.7830
Epoch 6/20
875/875 [==============================] - 13s 15ms/step - loss: 0.3509 - accuracy: 0.8676
Epoch 7/20
875/875 [==============================] - 9s 11ms/step - loss: 0.2614 - accuracy: 0.9053
Epoch 8/20
875/875 [==============================] - 10s 11ms/step - loss: 0.2173 - accuracy: 0.9246
Epoch 9/20
875/875 [==============================] - 9s 10ms/step - loss: 0.1887 - accuracy: 0.9360
Epoch 10/20
875/875 [==============================] - 9s 10ms/step - loss: 0.1646 - ac

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 1s 5ms/step - loss: 0.1232 - accuracy: 0.9611


[0.12315348535776138, 0.9611428380012512]

In [ ]:
from sklearn.metrics import f1_score

y_pred = model.predict(X_test)
y_pred_classes = tf.round(y_pred)

# Izračunavanje F1-score
f1 = f1_score(y_test, y_pred_classes)

219/219 [==============================] - 2s 7ms/step


In [ ]:
f1

0.961560203504805

Dosad najbolji model!
###**14.POKUŠAJ**:4 skrivena sloja,batch_size=32,broj epoha 20,optimizator adam,aktivacija sigmoid i relu u skrivenim slojevima

In [ ]:
from tensorflow.keras.layers import Dropout


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
povijest=model.fit(X_train, y_train, epochs=20,batch_size=32)


Epoch 1/20
875/875 [==============================] - 40s 43ms/step - loss: 0.2240 - accuracy: 0.9079
Epoch 2/20
875/875 [==============================] - 34s 39ms/step - loss: 0.1009 - accuracy: 0.9646
Epoch 3/20
875/875 [==============================] - 36s 41ms/step - loss: 0.0559 - accuracy: 0.9822
Epoch 4/20
875/875 [==============================] - 33s 38ms/step - loss: 0.0271 - accuracy: 0.9924
Epoch 5/20
875/875 [==============================] - 33s 38ms/step - loss: 0.0200 - accuracy: 0.9941
Epoch 6/20
875/875 [==============================] - 37s 42ms/step - loss: 0.0169 - accuracy: 0.9956
Epoch 7/20
875/875 [==============================] - 34s 39ms/step - loss: 0.0108 - accuracy: 0.9970
Epoch 8/20
875/875 [==============================] - 35s 40ms/step - loss: 0.0127 - accuracy: 0.9970
Epoch 9/20
875/875 [==============================] - 34s 39ms/step - loss: 0.0127 - accuracy: 0.9960
Epoch 10/20
875/875 [==============================] - 33s 38ms/step - loss: 0.009

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 2s 8ms/step - loss: 0.3549 - accuracy: 0.9579


[0.3548876643180847, 0.9578571319580078]

Dakle,zaključujemo da nam je najbolje rezultate dao 13.pokušaj u kojem smo imali 4 skrivena sloja,aktivacijsku funkciju relu u skrivenim slojevima i sigmoid u izlaznom sloju,optimizator sgd,binarnu krosentropiju.
Za očekivati je da bi još više skrivenih slojeva dalo još bolje rezultate,ali točnost je već sada jako visoka,gubitak dosta mali i mogu biti zadovoljna dobivenim rezultatima.
Također,naknadno sam izračunala i f1_score za ovaj slučaj jer je on dobra mjera kvalitete modela i dobila da je on 0.961,što je odličan rezultat.

###**REKURENTNE NEURONSKE MREŽE**

Započet ćemo s jednostavnom strukturom,slično onoj s vježbi.Može se primjetiti da je uvježbavanje dosta sporije nego kod unaprijednih neuronskih mreža zato ćemo za početak imati manji broj epoha i veći batch_size nego kod unaprijednih neuronskih mreža.
###**1.POKUŠAJ**:1 embedding,1 rekurentni sloj,sigmoid,binarna krosentropija,optimizator adam

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5000,
                                    output_dim=128
                                    ))
model.add(tf.keras.layers.SimpleRNN(128))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
povijest=model.fit(X_train, y_train, epochs=5,batch_size=128)


Epoch 1/5
219/219 [==============================] - 1250s 6s/step - loss: 0.6951 - accuracy: 0.4995
Epoch 2/5
219/219 [==============================] - 1236s 6s/step - loss: 0.6944 - accuracy: 0.5000
Epoch 3/5
219/219 [==============================] - 1235s 6s/step - loss: 0.6952 - accuracy: 0.4967
Epoch 4/5
219/219 [==============================] - 1232s 6s/step - loss: 0.6942 - accuracy: 0.4967
Epoch 5/5
219/219 [==============================] - 1235s 6s/step - loss: 0.6949 - accuracy: 0.5053


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 80s 366ms/step - loss: 0.6946 - accuracy: 0.5013


[0.6946445107460022, 0.50128573179245]

Vidimo da ovaj model daje jako malu točnost,a i izvršavao se jako dugo.Pokušat ćemo neke tehnike regularizacije.
###**2.POKUŠAJ**:1 embedding,1 rekurentni sloj,regularizacijasigmoid,binarna krosentropija,optimizator adam

In [ ]:
from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5000,
                                    output_dim=128
                                    ))
model.add(tf.keras.layers.SimpleRNN(128))
model.add(Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
povijest=model.fit(X_train, y_train, epochs=5,batch_size=128)


Epoch 1/5
219/219 [==============================] - 1235s 6s/step - loss: 0.7140 - accuracy: 0.4976
Epoch 2/5
219/219 [==============================] - 1236s 6s/step - loss: 0.7115 - accuracy: 0.5040
Epoch 3/5
219/219 [==============================] - 1237s 6s/step - loss: 0.7048 - accuracy: 0.5004
Epoch 4/5
219/219 [==============================] - 1238s 6s/step - loss: 0.7027 - accuracy: 0.4951
Epoch 5/5
219/219 [==============================] - 1232s 6s/step - loss: 0.6982 - accuracy: 0.5031


Ponovno loša točnost.
###**3.POKUŠAJ**:1 embedding,1 LSTM sloj,regularizacijasigmoid,binarna krosentropija,optimizator adam,parametar više

In [ ]:
from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5000,
                                    output_dim=128,input_length=5000
                                    ))
model.add(tf.keras.layers.LSTM(128))
model.add(Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=128)


Epoch 1/5
219/219 [==============================] - 64s 272ms/step - loss: 0.6935 - accuracy: 0.4984
Epoch 2/5
219/219 [==============================] - 61s 281ms/step - loss: 0.6933 - accuracy: 0.5050
Epoch 3/5
219/219 [==============================] - 61s 280ms/step - loss: 0.6933 - accuracy: 0.4996
Epoch 4/5
219/219 [==============================] - 62s 283ms/step - loss: 0.6932 - accuracy: 0.4961
Epoch 5/5
219/219 [==============================] - 64s 290ms/step - loss: 0.6932 - accuracy: 0.5010


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 18s 79ms/step - loss: 0.6934 - accuracy: 0.4987


[0.6933560371398926, 0.49871429800987244]

Točnost ponovno loša.
###**4.POKUŠAJ**:1 embedding,više LSTM slojeva,regularizacija,sigmoid,binarna krosentropija,optimizator adam,parametar više

In [ ]:
from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5000,
                                    output_dim=128,input_length=5000
                                    ))
model.add(tf.keras.layers.LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(tf.keras.layers.LSTM(64))
model.add(Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=128)


Epoch 1/5
219/219 [==============================] - 80s 346ms/step - loss: 0.6935 - accuracy: 0.4993
Epoch 2/5
219/219 [==============================] - 77s 354ms/step - loss: 0.6933 - accuracy: 0.5008
Epoch 3/5
219/219 [==============================] - 76s 348ms/step - loss: 0.6933 - accuracy: 0.4946
Epoch 4/5
219/219 [==============================] - 77s 350ms/step - loss: 0.6933 - accuracy: 0.5027
Epoch 5/5
219/219 [==============================] - 81s 368ms/step - loss: 0.6933 - accuracy: 0.4939


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 25s 111ms/step - loss: 0.6932 - accuracy: 0.5013


[0.6931518316268921, 0.50128573179245]

Još loše,ali malo bolje.
###**5.POKUŠAJ**:1 embedding,više LSTM slojeva,regularizacija sigmoid,binarna krosentropija,optimizator adam,parametar više,manji batch_size i više epoha

In [ ]:
from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5000,
                                    output_dim=128,input_length=5000
                                    ))
model.add(tf.keras.layers.LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(tf.keras.layers.LSTM(64))
model.add(Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
875/875 [==============================] - 221s 248ms/step - loss: 0.6937 - accuracy: 0.4989
Epoch 2/10
875/875 [==============================] - 216s 247ms/step - loss: 0.6933 - accuracy: 0.5010
Epoch 3/10
875/875 [==============================] - 215s 246ms/step - loss: 0.6934 - accuracy: 0.4990
Epoch 4/10
875/875 [==============================] - 215s 245ms/step - loss: 0.6933 - accuracy: 0.4986
Epoch 5/10
875/875 [==============================] - 216s 247ms/step - loss: 0.6932 - accuracy: 0.5033
Epoch 6/10
875/875 [==============================] - 217s 248ms/step - loss: 0.6933 - accuracy: 0.5012
Epoch 7/10
875/875 [==============================] - 216s 247ms/step - loss: 0.6933 - accuracy: 0.4990
Epoch 8/10
875/875 [==============================] - 218s 250ms/step - loss: 0.6933 - accuracy: 0.4934
Epoch 9/10
875/875 [==============================] - 223s 254ms/step - loss: 0.6933 - accuracy: 0.4985
Epoch 10/10
875/875 [==============================] - 219s 250m

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 26s 113ms/step - loss: 0.6931 - accuracy: 0.5013


[0.693145215511322, 0.50128573179245]

###**6.POKUŠAJ**:1 embedding,više GRU slojeva,regularizacijasigmoid,binarna krosentropija,optimizator adam,parametar više,manji batch_size i više epoha

In [ ]:
from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5000,
                                    output_dim=128,input_length=5000
                                    ))
model.add(tf.keras.layers.GRU(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(tf.keras.layers.GRU(64))
model.add(Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
875/875 [==============================] - 211s 234ms/step - loss: 0.6940 - accuracy: 0.4980
Epoch 2/10
875/875 [==============================] - 204s 233ms/step - loss: 0.6935 - accuracy: 0.5019
Epoch 3/10
875/875 [==============================] - 204s 233ms/step - loss: 0.6935 - accuracy: 0.4985
Epoch 4/10
875/875 [==============================] - 204s 233ms/step - loss: 0.6935 - accuracy: 0.4936
Epoch 5/10
875/875 [==============================] - 203s 232ms/step - loss: 0.6934 - accuracy: 0.5023
Epoch 6/10
875/875 [==============================] - 204s 233ms/step - loss: 0.6934 - accuracy: 0.5022
Epoch 7/10
875/875 [==============================] - 240s 274ms/step - loss: 0.6935 - accuracy: 0.4961
Epoch 8/10
875/875 [==============================] - 207s 237ms/step - loss: 0.6934 - accuracy: 0.4967
Epoch 9/10
875/875 [==============================] - 204s 233ms/step - loss: 0.6935 - accuracy: 0.4973
Epoch 10/10
875/875 [==============================] - 203s 232m

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 24s 105ms/step - loss: 0.6931 - accuracy: 0.5013


[0.6931454539299011, 0.50128573179245]

###**8.POKUŠAJ**:1 embedding,više LSTM slojeva,regularizacija sigmoid,binarna krosentropija,optimizator SGD,parametar više,manji batch_size i više epoha

In [ ]:
from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5000,
                                    output_dim=128,input_length=5000
                                    ))
model.add(tf.keras.layers.LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(tf.keras.layers.LSTM(64))
model.add(Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
875/875 [==============================] - 218s 245ms/step - loss: 0.6933 - accuracy: 0.4985
Epoch 2/10
875/875 [==============================] - 213s 243ms/step - loss: 0.6933 - accuracy: 0.4998
Epoch 3/10
875/875 [==============================] - 212s 242ms/step - loss: 0.6932 - accuracy: 0.5012
Epoch 4/10
365/875 [===========>..................] - ETA: 2:04 - loss: 0.6933 - accuracy: 0.4955

###**7.POKUŠAJ**:1 embedding,više BI-RNN slojeva,regularizacijasigmoid,binarna krosentropija,optimizator adam,parametar više,manji batch_size i više epoha

In [ ]:
from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5000,
                                    output_dim=128,input_length=5000
                                    ))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(64)))
model.add(Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
 42/875 [>.............................] - ETA: 4:34:24 - loss: 0.7259 - accuracy: 0.4836

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 295s 1s/step - loss: 0.7071 - accuracy: 0.4987


[0.7071397304534912, 0.49871429800987244]

###**9.POKUŠAJ**:1 embedding,više rnn slojeva,sigmoid,binarna krosentropija,optimizator adam,parametar više,manji batch_size i više epoha

In [ ]:
from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5000,
                                    output_dim=256
                                    ))
model.add(tf.keras.layers.SimpleRNN(256,return_sequences=True))
model.add(Dropout(0.5))
model.add(tf.keras.layers.SimpleRNN(128,return_sequences=True))
model.add(Dropout(0.5))
model.add(tf.keras.layers.SimpleRNN(64,return_sequences=True))
model.add(Dropout(0.5))
model.add(tf.keras.layers.SimpleRNN(32))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=128)


Epoch 1/10
 46/219 [=====>........................] - ETA: 1:14:08 - loss: 0.8823 - accuracy: 0.4988

###**10.POKUŠAJ**:1 embedding,sigmoid,binarna krosentropija,optimizator adam,parametar više,manji batch_size i više epoha,early stopping

In [ ]:
from keras.callbacks import EarlyStopping

from tensorflow.keras.layers import Dropout

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=5000,
                                    output_dim=128
                                    ))
model.add(tf.keras.layers.SimpleRNN(128))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
model.fit(X_train, y_train, epochs=5,  callbacks=[early_stopping])


Epoch 1/5
875/875 [==============================] - ETA: 0s - loss: 0.7494 - accuracy: 0.4986

875/875 [==============================] - 3736s 4s/step - loss: 0.7494 - accuracy: 0.4986
Epoch 2/5
875/875 [==============================] - ETA: 0s - loss: 0.7003 - accuracy: 0.5036

875/875 [==============================] - 3691s 4s/step - loss: 0.7003 - accuracy: 0.5036
Epoch 3/5
875/875 [==============================] - ETA: 0s - loss: 0.6967 - accuracy: 0.5002

875/875 [==============================] - 3769s 4s/step - loss: 0.6967 - accuracy: 0.5002
Epoch 4/5
875/875 [==============================] - ETA: 0s - loss: 0.6962 - accuracy: 0.4985

875/875 [==============================] - 3757s 4s/step - loss: 0.6962 - accuracy: 0.4985
Epoch 5/5
784/875 [=========================>....] - ETA: 6:37 - loss: 0.6958 - accuracy: 0.4998

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 122s 556ms/step - loss: 0.6936 - accuracy: 0.5013


[0.6936230063438416, 0.50128573179245]

###**11.POKUŠAJ**:1 embedding,više birnn slojeva,sigmoid,binarna krosentropija,optimizator adam,parametar više,manji batch_size i više epoha,early stopping,batch

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(64,input_shape=(X_train.shape[1],1), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(tf.keras.layers.LSTM(64, return_sequences=True
                                    ))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(units=32, activation='relu'))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


NameError: name 'model' is not defined

Vidimo da ni s jednim modelom nisam uspjela značajnije povećati točnost.Moguće da bih korištenjem transfer learninga ili fine tuninga dobila bolje rezultate,ali sam u međuvremenu vidjela da konvolucijske neuronske mreže daju veliku točnost pa sam se posvetila njima.

#**KONVOLUCIJSKE NEURONSKE MREŽE**
Budući da nam rekurentne neuronske mreže nisu dale jako dobre rezultate,pokušat ćemo vidjeti kakve rezultate bi dale konvolucijske neuronske mreže.Iako se one najviše koriste za slike,možda bi mogle dati dobre rezultate u našem slučaju(1D konvolucije pogodne za tekst).

###**1.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija(za početak)

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(32, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16, activation = 'relu'))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 259s 292ms/step - loss: 0.2544 - accuracy: 0.8929
Epoch 2/5
875/875 [==============================] - 249s 285ms/step - loss: 0.1441 - accuracy: 0.9459
Epoch 3/5
875/875 [==============================] - 227s 260ms/step - loss: 0.1029 - accuracy: 0.9615
Epoch 4/5
875/875 [==============================] - 216s 247ms/step - loss: 0.0719 - accuracy: 0.9735
Epoch 5/5
875/875 [==============================] - 215s 246ms/step - loss: 0.0468 - accuracy: 0.9833


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 17s 71ms/step - loss: 0.1799 - accuracy: 0.9474


[0.1799173802137375, 0.9474285840988159]

Vidimo da smo zapravo na prvom pokušaju dobili jako visoku točnost.Pokušat ćemo je sad još povećati,logičan pokušaj je regularizacija jer nam se točnost smanjila na skupu za testiranje.
###**2.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija(za početak),regularizacija

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense
from tensorflow.keras import regularizers



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(32, kernel_size=5,input_shape=(X_train.shape[1],1),kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16, activation = 'relu'))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 225s 256ms/step - loss: 0.3427 - accuracy: 0.8812
Epoch 2/5
875/875 [==============================] - 221s 252ms/step - loss: 0.2284 - accuracy: 0.9276
Epoch 3/5
875/875 [==============================] - 227s 259ms/step - loss: 0.2012 - accuracy: 0.9374
Epoch 4/5
875/875 [==============================] - 219s 250ms/step - loss: 0.1827 - accuracy: 0.9448
Epoch 5/5
875/875 [==============================] - 217s 248ms/step - loss: 0.1655 - accuracy: 0.9512


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 15s 69ms/step - loss: 0.1754 - accuracy: 0.9457


[0.17536437511444092, 0.9457142949104309]

Dobili smo čak nešto lošiju točnost i nešto niži loss,ali i dalje odlične brojke.

###**3.POKUŠAJ**:jednostavan model,5 epoha,sgd,binarna krosentropija(za početak),bez regularizacije

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(32, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16, activation = 'relu'))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 226s 257ms/step - loss: 0.6906 - accuracy: 0.5510
Epoch 2/5
875/875 [==============================] - 213s 244ms/step - loss: 0.6811 - accuracy: 0.7006
Epoch 3/5
875/875 [==============================] - 213s 243ms/step - loss: 0.6520 - accuracy: 0.7862
Epoch 4/5
875/875 [==============================] - 214s 244ms/step - loss: 0.5291 - accuracy: 0.8226
Epoch 5/5
875/875 [==============================] - 233s 266ms/step - loss: 0.3576 - accuracy: 0.8527


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 20s 85ms/step - loss: 0.3181 - accuracy: 0.8659


[0.31809699535369873, 0.8658571243286133]

Vidimo da smo dobili lošije rezultate ovim optimizatorom.

###**4.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija,regularizacija(droput ovaj put)

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(32, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 231s 260ms/step - loss: 0.3306 - accuracy: 0.8482
Epoch 2/5
875/875 [==============================] - 233s 267ms/step - loss: 0.2091 - accuracy: 0.9120
Epoch 3/5
875/875 [==============================] - 228s 261ms/step - loss: 0.1575 - accuracy: 0.9344
Epoch 4/5
875/875 [==============================] - 228s 260ms/step - loss: 0.1298 - accuracy: 0.9483
Epoch 5/5
875/875 [==============================] - 225s 258ms/step - loss: 0.1122 - accuracy: 0.9544


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 15s 66ms/step - loss: 0.1265 - accuracy: 0.9557


[0.12654374539852142, 0.9557142853736877]

Na ovaj način smo još više povećali točnost i smanjili loss funkciju.

###**5.POKUŠAJ**:jednostavan model,10 epoha,adam,binarna krosentropija(za početak),regularizacija(droput ovaj put)

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(32, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10)


Epoch 1/10
875/875 [==============================] - 226s 257ms/step - loss: 0.3138 - accuracy: 0.8639
Epoch 2/10
875/875 [==============================] - 219s 251ms/step - loss: 0.1929 - accuracy: 0.9244
Epoch 3/10
875/875 [==============================] - 215s 245ms/step - loss: 0.1447 - accuracy: 0.9421
Epoch 4/10
875/875 [==============================] - 215s 246ms/step - loss: 0.1229 - accuracy: 0.9500
Epoch 5/10
875/875 [==============================] - 214s 245ms/step - loss: 0.1035 - accuracy: 0.9581
Epoch 6/10
875/875 [==============================] - 213s 243ms/step - loss: 0.0894 - accuracy: 0.9616
Epoch 7/10
875/875 [==============================] - 214s 244ms/step - loss: 0.0783 - accuracy: 0.9663
Epoch 8/10
875/875 [==============================] - 211s 242ms/step - loss: 0.0709 - accuracy: 0.9687
Epoch 9/10
875/875 [==============================] - 213s 243ms/step - loss: 0.0645 - accuracy: 0.9705
Epoch 10/10
875/875 [==============================] - 212s 243m

In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 14s 62ms/step - loss: 0.1540 - accuracy: 0.9577


[0.15399624407291412, 0.9577142596244812]

Vidimo da smo dobili nešto veću točnost,ali je i gubitak veći od prethodnog pokušaja.

###**6.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija(za početak),regularizacija(droput ovaj put),promjenjiva stopa učenja

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense
from tensorflow.keras.callbacks import LearningRateScheduler

def exponential_decay(epoch):
    initial_lr = 0.1
    k = 0.1
    lr = initial_lr * np.exp(-k * epoch)
    return lr


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(32, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

lr_scheduler = LearningRateScheduler(exponential_decay)


In [ ]:
history = model.fit(X_train, y_train, epochs=5, callbacks=[lr_scheduler])


Epoch 1/5
875/875 [==============================] - 265s 298ms/step - loss: 1.0150 - accuracy: 0.4987 - lr: 0.1000
Epoch 2/5
875/875 [==============================] - 259s 296ms/step - loss: 0.6949 - accuracy: 0.5056 - lr: 0.0905
Epoch 3/5
875/875 [==============================] - 263s 301ms/step - loss: 0.6945 - accuracy: 0.5035 - lr: 0.0819
Epoch 4/5
875/875 [==============================] - 314s 359ms/step - loss: 0.6946 - accuracy: 0.5010 - lr: 0.0741
Epoch 5/5
875/875 [==============================] - 245s 280ms/step - loss: 0.6945 - accuracy: 0.4950 - lr: 0.0670


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 17s 79ms/step - loss: 0.6961 - accuracy: 0.4987


[0.6961305141448975, 0.49871429800987244]

Vidimo da su rezultati prilično loši.Zapravo,adam optimizator već ima ugrađeno individualno biranje stope učenja pa zapravo na ovaj način samo radimo štetu.

###**7.POKUŠAJ**:jednostavan model, epoha,adadelta,binarna krosentropija(za početak),regularizacija(droput ovaj put),promjenjiva stopa učenja

In [ ]:
from tensorflow.keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(32, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

optimizer = Adadelta()

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 256s 290ms/step - loss: 0.6930 - accuracy: 0.5292
Epoch 2/5
875/875 [==============================] - 256s 292ms/step - loss: 0.6926 - accuracy: 0.5650
Epoch 3/5
875/875 [==============================] - 259s 295ms/step - loss: 0.6920 - accuracy: 0.5895
Epoch 4/5
875/875 [==============================] - 265s 302ms/step - loss: 0.6913 - accuracy: 0.6071
Epoch 5/5
875/875 [==============================] - 266s 305ms/step - loss: 0.6907 - accuracy: 0.6130


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 18s 81ms/step - loss: 0.6902 - accuracy: 0.6664


[0.69020015001297, 0.6664285659790039]

Ovaj optimizator daje lošije rezultate.

###**8.POKUŠAJ**:jednostavan model, epoha,rmsprop,binarna krosentropija(za početak),regularizacija(droput ovaj put),promjenjiva stopa učenja

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(32, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

optimizer = RMSprop()

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 245s 279ms/step - loss: 0.3363 - accuracy: 0.8531
Epoch 2/5
875/875 [==============================] - 244s 279ms/step - loss: 0.2205 - accuracy: 0.9163
Epoch 3/5
875/875 [==============================] - 244s 278ms/step - loss: 0.1823 - accuracy: 0.9311
Epoch 4/5
875/875 [==============================] - 243s 277ms/step - loss: 0.1599 - accuracy: 0.9397
Epoch 5/5
875/875 [==============================] - 250s 286ms/step - loss: 0.1481 - accuracy: 0.9456


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 16s 71ms/step - loss: 0.1446 - accuracy: 0.9520


[0.1446104198694229, 0.9520000219345093]

Vidimo da smo dobili dosta dobre rezultate,ali nešto gore nego s adam optimizatorom.

###**9.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija(za početak),regularizacija(droput i batch normalizacija),promjenjiva stopa učenja

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(32, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(BatchNormalization())
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(BatchNormalization())

model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 423s 480ms/step - loss: 0.3006 - accuracy: 0.8805
Epoch 2/5
875/875 [==============================] - 384s 439ms/step - loss: 0.1991 - accuracy: 0.9209
Epoch 3/5
875/875 [==============================] - 383s 437ms/step - loss: 0.1713 - accuracy: 0.9296
Epoch 4/5
875/875 [==============================] - 395s 452ms/step - loss: 0.1435 - accuracy: 0.9394
Epoch 5/5
875/875 [==============================] - 398s 455ms/step - loss: 0.1416 - accuracy: 0.9416


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 21s 96ms/step - loss: 0.4335 - accuracy: 0.8894


[0.4334709346294403, 0.8894285559654236]

Vidimo da batch normalizacija daje lošije rezultate.

###**10.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija(za početak),regularizacija(droput),veći broj filtera u slojevima

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(64, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(128, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))



model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 667s 753ms/step - loss: 0.2688 - accuracy: 0.8871
Epoch 2/5
875/875 [==============================] - 648s 740ms/step - loss: 0.1412 - accuracy: 0.9463
Epoch 3/5
875/875 [==============================] - 638s 729ms/step - loss: 0.1050 - accuracy: 0.9605
Epoch 4/5
875/875 [==============================] - 622s 711ms/step - loss: 0.0792 - accuracy: 0.9711
Epoch 5/5
875/875 [==============================] - 621s 710ms/step - loss: 0.0648 - accuracy: 0.9758


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 40s 180ms/step - loss: 0.1261 - accuracy: 0.9569


[0.1261330544948578, 0.9568571448326111]

Dobili smo najveću točnost do sada.

###**11.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija(za početak),regularizacija(droput),veći broj filtera u slojevima,još jedan konvolucijskei sloj

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(64, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(128, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(tf.keras.layers.Conv1D(256, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))



model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 879s 1s/step - loss: 0.3216 - accuracy: 0.8605
Epoch 2/5
875/875 [==============================] - 868s 992ms/step - loss: 0.1894 - accuracy: 0.9265
Epoch 3/5
875/875 [==============================] - 878s 1s/step - loss: 0.1414 - accuracy: 0.9461
Epoch 4/5
875/875 [==============================] - 861s 984ms/step - loss: 0.1154 - accuracy: 0.9550
Epoch 5/5
875/875 [==============================] - 879s 1s/step - loss: 0.0950 - accuracy: 0.9640


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 63s 287ms/step - loss: 0.1568 - accuracy: 0.9456


[0.1568034291267395, 0.9455714225769043]

Vidimo da smo dobili nešto lošije rezultate.

**12.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija(za početak),regularizacija(droput),veći broj filtera u slojevima,average pooling

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense,AveragePooling1D



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(64, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.AveragePooling1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(128, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.AveragePooling1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))



model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 359s 408ms/step - loss: 0.2949 - accuracy: 0.8794
Epoch 2/5
875/875 [==============================] - 360s 411ms/step - loss: 0.1767 - accuracy: 0.9313
Epoch 3/5
875/875 [==============================] - 353s 403ms/step - loss: 0.1342 - accuracy: 0.9489
Epoch 4/5
875/875 [==============================] - 353s 403ms/step - loss: 0.1064 - accuracy: 0.9601
Epoch 5/5
875/875 [==============================] - 353s 404ms/step - loss: 0.0841 - accuracy: 0.9686


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 26s 117ms/step - loss: 0.1285 - accuracy: 0.9549


[0.12849631905555725, 0.9548571705818176]

Dosta dobri rezultati,nešto niža točnost od najbolje točnosti,ali je loss funkcija manja.

**13.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija(za početak),regularizacija(droput),veći broj filtera u slojevima,kernel size=3

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(64, kernel_size=3,input_shape=(X_train.shape[1],1), activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))



model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 532s 603ms/step - loss: 0.2725 - accuracy: 0.8851
Epoch 2/5
875/875 [==============================] - 551s 629ms/step - loss: 0.1468 - accuracy: 0.9452
Epoch 3/5
875/875 [==============================] - 538s 615ms/step - loss: 0.1025 - accuracy: 0.9606
Epoch 4/5
875/875 [==============================] - 524s 600ms/step - loss: 0.0772 - accuracy: 0.9709
Epoch 5/5
875/875 [==============================] - 549s 627ms/step - loss: 0.0617 - accuracy: 0.9763


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 35s 158ms/step - loss: 0.1417 - accuracy: 0.9564


[0.14169591665267944, 0.956428587436676]

Ipak nešto lošija točnost nego prije.

**14.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija(za početak),regularizacija(droput),veći broj filtera u slojevima,padding=same

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(64, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(128, kernel_size=5, activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))



model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 458s 522ms/step - loss: 0.2757 - accuracy: 0.8840
Epoch 2/5
875/875 [==============================] - 417s 477ms/step - loss: 0.1512 - accuracy: 0.9425
Epoch 3/5
875/875 [==============================] - 409s 468ms/step - loss: 0.1062 - accuracy: 0.9607
Epoch 4/5
875/875 [==============================] - 410s 469ms/step - loss: 0.0835 - accuracy: 0.9698
Epoch 5/5
875/875 [==============================] - 403s 461ms/step - loss: 0.0644 - accuracy: 0.9758


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 25s 116ms/step - loss: 0.1342 - accuracy: 0.9600


[0.1342094987630844, 0.9599999785423279]

Dobili smo najbolju točnost i najmanji gubitak!

**15.POKUŠAJ**:jednostavan model,5 epoha,adam,binarna krosentropija(za početak),regularizacija(dropout),veći broj filtera u slojevima,padding=same,strides=2

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,BatchNormalization,Dense



model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(64, kernel_size=5,input_shape=(X_train.shape[1],1), activation='relu',padding='same',strides=2))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))                                                               # pool_size - dimenzija pooling okvira, po defaultu strides=pool_size
model.add(tf.keras.layers.Conv1D(128, kernel_size=5, activation='relu',padding='same',strides=2))
model.add(tf.keras.layers.MaxPool1D(pool_size=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(32, activation = 'relu'))
model.add(Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))



model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5)


Epoch 1/5
875/875 [==============================] - 205s 234ms/step - loss: 0.0867 - accuracy: 0.9674
Epoch 2/5
875/875 [==============================] - 206s 236ms/step - loss: 0.0732 - accuracy: 0.9722
Epoch 3/5
875/875 [==============================] - 204s 234ms/step - loss: 0.0655 - accuracy: 0.9757
Epoch 4/5
875/875 [==============================] - 203s 232ms/step - loss: 0.0608 - accuracy: 0.9762
Epoch 5/5
875/875 [==============================] - 231s 264ms/step - loss: 0.0580 - accuracy: 0.9780


In [ ]:
model.evaluate(X_test, y_test)


219/219 [==============================] - 14s 65ms/step - loss: 0.1719 - accuracy: 0.9519


[0.17190752923488617, 0.9518571496009827]

Vidimo da je bolja točnost u prethodnom primjeru.

Najbolju točnost dobili smo u 14.pokušaju.Sada ćemo izračunati f1_score tog modela kako bismo ga mogli usporediti s onim kod unaprijednih neuronskih mreža i na taj način definitivno zaključiti koji je najbolji model.

In [ ]:
from sklearn.metrics import f1_score

y_pred = model.predict(X_test)
y_pred_classes = tf.round(y_pred)

# Izračunavanje F1-score
f1 = f1_score(y_test, y_pred_classes)
f1

219/219 [==============================] - 26s 118ms/step


0.9601933466022179

Dakle,vidimo da smo ponovno dobili jako velik f1 score te zaključujemo da je i ovaj model jako uspješan za naš primjer.Možda je nešto uspješniji model s unaprijednim neuronskim mrežama,ali razlike su jako male pa mislim da ne bismo pogriješili koji god model odaberemo.